Johannes' code to perform the IDM scans

In [1]:
# -*- coding: utf-8 -*-
from anyBSM import anyBSM
import numpy as np
import pandas as pd
import scipy
import scipy.special as spe
import os
import sys
import matplotlib.pyplot as plt
import subprocess
from matplotlib import colors
import math
import cmath
import nlounit as pu
import ewpo
import dm
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
from anyBSM.loopfunctions import C0
import random
from anyBSM.loopfunctions import B0, C0, dB0, A0
from anyBSM.plotting import setAnyStyle
import anyBSM.plotting as plotting
setAnyStyle()
import time

In [2]:
SM = anyBSM(
    'SM', # load the SM
    scheme_name = "OStadpoles", # load the OS scheme
    progress = False, # dont show any progress bars
    caching = 2, # enable the cache
    quiet=True
)
IDM = anyBSM(
    'IDM', # load the SM
    scheme_name = "OStadpoles", # load the OS scheme
    progress = False, # dont show any progress bars
    caching = 2, # enable the cache
    quiet=True
)

particle_exclusion = [str(p) for p in list(IDM.all_particles.values()) if str(p) not in ['h2','Ah2','Hm2']]
SM.set_evaluation_mode('numerical')
IDM.set_evaluation_mode('numerical')

In [11]:
Mh = 125.1
Nc=3
Qt=2/3
Qb=-1/3
Qtau=-1
lamQCD5=210*10**(-3)
mHmin = 55
mHmax  = 1200
mAmax  = 1200
mHpmax = 1200
vev=246.21965
lam1 = Mh**2/vev**2
MW=IDM.parameters['MWp'].nvalue
MZ=IDM.parameters['MZ'].nvalue
Mt=IDM.parameters['Mu3'].nvalue
Mb=IDM.parameters['Md3'].nvalue
Mtau=IDM.parameters['Me3'].nvalue
#Mb=4.78#IDM.parameters['Md3'].nvalue
#Mtau=1.77686
#IDM.parameters['vSM'].nvalue
alphaem=1/137.035999084
eL = np.sqrt(4*np.pi*alphaem)
cw=MW/MZ
sw=math.sqrt(1-cw**2)
eps=10e-4
_MWMZaEMscheme = 1 # to switch between GF and (MW,MZ,aEM) EW schemes

def alphas(lam_QCD,mu,N_af):
     #double precision, intent(in):: mu,N_af,lam_QCD
     #double precision:: beta0,beta1,beta2,ell_mu
     beta0=11-2/3*N_af
     beta1=51-19/3*N_af
     beta2=2857-5033/9*N_af+325/27*N_af**2
     ell_mu=np.log(mu**2/lam_QCD**2)
     return 4*np.pi/beta0/ell_mu*(1-2*beta1/beta0**2*np.log(ell_mu)/ell_mu+4*beta1**2/beta0**4/ell_mu**2*((np.log(ell_mu)-1/2)**2+beta2*beta0/8/beta1**2-5/4))


# h->gamgam results
muQCD=Mh/2
I1t=-2*Nc*Qt**2*Mt**2/Mh**2*(1-1/2*Mh**2*(1-4*Mt**2/Mh**2)*C0(0,0,Mh**2,Mt**2,Mt**2,Mt**2))
I1tapprox=-Nc*Qt**2/3
I1b=-2*Nc*Qb**2*Mb**2/Mh**2*(1-1/2*Mh**2*(1-4*Mb**2/Mh**2)*C0(0,0,Mh**2,Mb**2,Mb**2,Mb**2))
I1tau=-2*Qtau**2*Mtau**2/Mh**2*(1-1/2*Mh**2*(1-4*Mtau**2/Mh**2)*C0(0,0,Mh**2,Mtau**2,Mtau**2,Mtau**2))
I1W=1/2+3*MW**2/Mh**2-3*MW**2*(1-2*MW**2/Mh**2)*C0(0,0,Mh**2,MW**2,MW**2,MW**2)
I1Wapprox=7/4
def I1Hp(MHp,mu2):
    return (MHp**2-mu2**2)/Mh**2*(1+2*MHp**2*C0(0,0,Mh**2,MHp**2,MHp**2,MHp**2))
def I1Hpapprox(MHp,mu2):
    return -1/12*(1-mu2**2/MHp**2)
def SigphhBSM(MH,MA,MHp,mu2):
    res =MH**2*(1-mu2**2/MH**2)**2
    res+=MA**2*(1-mu2**2/MA**2)**2
    res+=2*MHp**2*(1-mu2**2/MHp**2)**2
    return -res/(48*np.pi**2*vev**2)
def F(x,y):
    if np.abs(x-y)<eps:
        return 0
    else:
        return (x**2-y**2+2*x*y*cmath.log(y/x))/(x-y)
def deltavovervBSM(MH,MA,MHp):
    res =-cw**2/sw**2*F(MH**2,MA**2)
    res+=(1-2*sw**2)/sw**2*(F(MHp**2,MH**2)+F(MHp**2,MA**2)) 
    return res/(64*np.pi**2*vev**2)
def DeltarBSM(MH,MA,MHp):
    res = F(MH**2,MA**2)-F(MH**2,MHp**2)-F(MA**2,MHp**2)
    return cw**2*res/(32*np.pi**2*sw**2*vev**2)
I2NLOQCD=I1t*(-alphas(lamQCD5,muQCD,5)/np.pi)
I3NNLOQCD=I1t*(-alphas(lamQCD5,muQCD,5)**2/np.pi**2)*(31/24-7/4*np.log(Mt**2/muQCD**2))
I2NLOEWold=-(I1t+I1W)*0.008
I2NLOEW=alphaem/(16*np.pi*sw**2)*(-24.1)
def I2BSMl3sq(MHp,mu2):
    return -1/(96*np.pi**2*vev**2)*(1-mu2**2/MHp**2)**2*(2*MHp**2+mu2**2)
def I2BSMl4pm5sq(MN,MHp,mu2):
    if np.abs(MN-MHp)<eps:
        return 0
    else:
        # 7/8/2024: change to new expressions, including gauge bosons (in BFM)
        term = -MHp**2*(1-MN**2/MHp**2)**2+mu2**2*(-38-MN**4/MHp**4+3*MN**2/MHp**2)+mu2**2*(17*MN**2+19*MHp**2)/(MN**2-MHp**2)*math.log(MN**2/MHp**2)+mu2**2*(1-MN**2/MHp**2)**3*cmath.log(1-MHp**2/MN**2)
        return term/(192*np.pi**2*vev**2)
def I2BSMl2(MH,MA,MHp,mu2,lam2):
    return -lam2/(384*np.pi**2*MHp**2)*(MH**2+MA**2+4*MHp**2-6*mu2**2)
def I2BSMl2PDOS(MH,MA,MHp,mu2,lam2):
    return -lam2*(-MH**2 + MHp**2 + mu2**2*math.log(MH**2) - mu2**2*math.log(MHp**2))/(192.*MHp**2*np.pi**2)
def I2BSMPDOS_F(MH,MA,MHp,mu2):
    return (7*Mt**2*(MH**2 - mu2**2))/(384.*MHp**2*np.pi**2*vev**2)
def I2BSMPDOS_S(MH,MA,MHp,mu2):
    res  = -(MA**2*(MH**2 - mu2**2)*math.log(MA**2/MHp**2))/(384.*(MA**2 - MHp**2)*np.pi**2*vev**2)
    res += -(MH**2*(MH**2 - mu2**2)*math.log(MH**2/MHp**2))/(384.*(MH**2 - MHp**2)*np.pi**2*vev**2)
    res += ((MH**2 - MHp**2)**3*mu2**2*cmath.log(1 - MH**2/MHp**2))/(96.*MH**4*MHp**2*np.pi**2*vev**2)
    res += ((-MA**2 + MH**2)**3*mu2**2*cmath.log(1 - MH**2/MA**2))/(192.*MH**4*MHp**2*np.pi**2*vev**2)
    temp = 12*MA**6*MHp**2 - 23*MA**4*MH**2*MHp**2 - 2*MH**2*MHp**2*mu2**4 + MA**2*(85*MH**4*MHp**2 + 24*MHp**6 + 46*MHp**2*mu2**4 - 2*MH**2*(23*MHp**4 + 40*MHp**2*mu2**2 + 2*mu2**4))
    res += (MH**2 - mu2**2)*temp/(2304*MA**2*MH**2*MHp**4*np.pi**2*vev**2)
    return res
def I2extlegVEVDr(MH,MA,MHp,mu2): # 1/8/2024: new expressions
    term = (-2 - MA**2/MHp**2 - MH**2/MHp**2 - (16*mu2**2)/MHp**2 + (4*mu2**4)/MHp**4 + (2*mu2**4)/(MA**2*MHp**2) + (2*mu2**4)/(MH**2*MHp**2) + (6*MA**2*math.log(MA**2/MHp**2))/(MA**2 - MHp**2) + (6*MH**2*math.log(MH**2/MHp**2))/(MH**2 - MHp**2)) 
    return term*(MHp**2-mu2**2)/(2304*np.pi**2*vev**2)
def I2rem(MH,MA,MHp,mu2):
    return I2extlegVEVDr(MH,MA,MHp,mu2)+I2BSMPDOS_S(MH,MA,MHp,mu2)
def I2F(MH,MA,MHp,mu2): 
    return 7*Mt**2/(32*np.pi**2*vev**2)*I1Hpapprox(MHp,mu2)+(I1tapprox+0*I1b+0*I1tau)*(1/2*SigphhBSM(MH,MA,MHp,mu2)-deltavovervBSM(MH,MA,MHp)-1/2*DeltarBSM(MH,MA,MHp))
def I2F_PDOS(MH,MA,MHp,mu2):
    return I2F(MH,MA,MHp,mu2)+I2BSMPDOS_F(MH,MA,MHp,mu2)
def I2V(MH,MA,MHp,mu2):
    return I1Wapprox*(1/2*SigphhBSM(MH,MA,MHp,mu2)-deltavovervBSM(MH,MA,MHp)-1/2*DeltarBSM(MH,MA,MHp))


@np.vectorize
def kappagam1L(MHp,mu2):
    return (np.sqrt((np.abs(I1t+I1W+I1Hp(MHp,mu2)+I2NLOQCD+I3NNLOQCD+I2NLOEW)**2)/(np.abs(I1t+I1W+I2NLOQCD+I3NNLOQCD+I2NLOEW))**2)-1)*100

@np.vectorize
def kappagam2L(MHp,MH,MA,mu2,lam2):
    return (np.sqrt((np.abs(I1t+I1W+I1Hp(MHp,mu2)+I2BSMl3sq(MHp,mu2)+I2BSMl4pm5sq(MH,MHp,mu2)+I2BSMl4pm5sq(MA,MHp,mu2)+I2BSMl2(MH,MA,MHp,mu2,lam2)+I2extlegVEVDr(MH,MA,MHp,mu2)+I2F(MH,MA,MHp,mu2)+I2V(MH,MA,MHp,mu2)+I2NLOQCD+I3NNLOQCD+I2NLOEW)**2)/(np.abs(I1t+I1W+I2NLOQCD+I3NNLOQCD+I2NLOEW))**2)-1)*100

@np.vectorize
def kappagam2LPDOS(MHp,MH,MA,mu2,lam2):
    return (np.sqrt((np.abs(I1t+I1W+I1Hp(MHp,mu2)+I2BSMl3sq(MHp,mu2)+I2BSMl4pm5sq(MH,MHp,mu2)+I2BSMl4pm5sq(MA,MHp,mu2)+I2BSMl2PDOS(MH,MA,MHp,mu2,lam2)+I2rem(MH,MA,MHp,mu2)+I2F_PDOS(MH,MA,MHp,mu2)+I2V(MH,MA,MHp,mu2)+I2NLOQCD+I3NNLOQCD+I2NLOEW)**2)/(np.abs(I1t+I1W+I2NLOQCD+I3NNLOQCD+I2NLOEW))**2)-1)*100

@np.vectorize
def DeltaGammagamgam1L(MHp,mu2):
    return (np.abs(I1t+I1W+I1Hp(MHp,mu2)+I2NLOQCD+I3NNLOQCD+I2NLOEW)**2)/(np.abs(I1t+I1W+I2NLOQCD+I3NNLOQCD+I2NLOEW))**2-1

@np.vectorize
def DeltaGammagamgam2L(MHp,MH,MA,mu2,lam2):
    return (np.abs(I1t+I1W+I1Hp(MHp,mu2)+I2BSMl3sq(MHp,mu2)+I2BSMl4pm5sq(MH,MHp,mu2)+I2BSMl4pm5sq(MA,MHp,mu2)+I2BSMl2(MH,MA,MHp,mu2,lam2)+I2extlegVEVDr(MH,MA,MHp,mu2)+I2F(MH,MA,MHp,mu2)+I2V(MH,MA,MHp,mu2)+I2NLOQCD+I3NNLOQCD+I2NLOEW)**2)/(np.abs(I1t+I1W+I2NLOQCD+I3NNLOQCD+I2NLOEW))**2-1

@np.vectorize
def DeltaGammagamgam2LPDOS(MHp,MH,MA,mu2,lam2):
    return (np.abs(I1t+I1W+I1Hp(MHp,mu2)+I2BSMl3sq(MHp,mu2)+I2BSMl4pm5sq(MH,MHp,mu2)+I2BSMl4pm5sq(MA,MHp,mu2)+I2BSMl2PDOS(MH,MA,MHp,mu2,lam2)+I2rem(MH,MA,MHp,mu2)+I2F_PDOS(MH,MA,MHp,mu2)+I2V(MH,MA,MHp,mu2)+I2NLOQCD+I3NNLOQCD+I2NLOEW)**2)/(np.abs(I1t+I1W+I2NLOQCD+I3NNLOQCD+I2NLOEW))**2-1


vt = (1/2 - 2*Qt*sw**2)/2
vb = (-1/2 - 2*Qb*sw**2)/2
vtau = (-1/2 -2*Qtau*sw**2)/2
J1W = 2*MW**2/(sw*cw*(Mh**2-MZ**2))*((cw**2*(5+Mh**2/(2*MW**2))-sw**2*(1.+Mh**2/(2.*MW**2)))*(1.+2.*MW**2*C0(0.,MZ**2,Mh**2,MW**2,MW**2,MW**2)+MZ**2/(Mh**2-MZ**2)*(B0(Mh**2,MW**2,MW**2)-B0(MZ**2,MW**2,MW**2)))-6.*cw**2*(Mh**2-MZ**2)*C0(0.,MZ**2,Mh**2,MW**2,MW**2,MW**2)+2.*sw**2*(Mh**2-MZ**2)*C0(0.,MZ**2,Mh**2,MW**2,MW**2,MW**2))
def J1f(mf):
    return (-4*mf**2/(sw*cw*(Mh**2-MZ**2))*(2.+(4.*mf**2-Mh**2+MZ**2)*C0(0.,MZ**2,Mh**2,mf**2,mf**2,mf**2)+2.*MZ**2/(Mh**2-MZ**2)*(B0(Mh**2,mf**2,mf**2)-B0(MZ**2,mf**2,mf**2))))
J1t = 3*Qt*vt*J1f(Mt)
def J1Hp(mHp,mu2):
    gHpHmZ = eL/(cw*sw)*(1/2-sw**2)
    lamHpHmh = 2*(mHp**2-mu2**2)/vev**2
    prefacJS = 2*vev**2/eL/(Mh**2-MZ**2)
    JS  = 1 + 2*mHp**2*C0(0,MZ**2,Mh**2,mHp**2,mHp**2,mHp**2)
    JS += MZ**2/(Mh**2-MZ**2)*(B0(Mh**2,mHp**2,mHp**2)-B0(MZ**2,mHp**2,mHp**2))
    return gHpHmZ*lamHpHmh*prefacJS*JS
J2NLOQCD=J1t*(-alphas(lamQCD5,muQCD,5)/np.pi)

@np.vectorize
def kappagamZ1L(MHp,mu2):
    return (np.sqrt((np.abs(J1t+J1W+J1Hp(MHp,mu2)+J2NLOQCD)**2)/(np.abs(J1t+J1W+J2NLOQCD))**2)-1)*100

@np.vectorize
def DeltaGammaZgam(MHp,mu2):
    return (np.abs(J1t+J1W+J1Hp(MHp,mu2)+J2NLOQCD)**2)/(np.abs(J1t+J1W+J2NLOQCD))**2-1

# Leading SM contributions to hhh
lamhhh0L=3*Mh**2/vev
d1lamhhh_SM = -48*Mt**4/(16*np.pi**2*vev**3)
myalphas=0.1179  
d2lamhhh_SM = 72*Mt**4/(256*np.pi**4*vev**3)*(64*np.pi*myalphas-13*Mt**2/vev**2)- _MWMZaEMscheme*(27 * cw**2 * Mt**6) / (32 * np.pi**4 * sw**2 * vev**5)

# Full 1L results for \lambda_{hhh} in SM and IDM, from anyH3
lamhhh0L_SM=SM.lambdahhh()['treelevel']
lamhhh1L_SM=SM.lambdahhh()['total']
@np.vectorize
def kappalam1L_fullIDM(mH,mA,mHp,mu2):
    return IDM.lambdahhh(parameters={'Mh2':mH,'MAh2':mA,'MHm2':mHp,'mHu2':mu2**2})['total']/lamhhh0L_SM

# IDM results for \lambda_{hhh} and \kappa_\lambda in the heavy Higgs (HH) scenario
def d1lamhhh_IDM_Honly(mH,mA,mHp,mu2):
    return (4*mH**4*(1-mu2**2/mH**2)**3)/(16*np.pi**2*vev**3)
def d1lamhhh_IDM_Aonly(mH,mA,mHp,mu2):
    return (4*mA**4*(1-mu2**2/mA**2)**3)/(16*np.pi**2*vev**3)
def d1lamhhh_IDM_Hpmonly(mH,mA,mHp,mu2):
    return (8*mHp**4*(1-mu2**2/mHp**2)**3)/(16*np.pi**2*vev**3)
def d1lamhhh_IDM_HH(mH,mA,mHp,mu2):
    return (4*mH**4*(1-mu2**2/mH**2)**3+4*mA**4*(1-mu2**2/mA**2)**3+8*mHp**4*(1-mu2**2/mHp**2)**3)/(16*np.pi**2*vev**3)
def d2lamhhh_IDM_HH_SS_decoup(mH,mA,mHp,mu2,l2,Q2): # NB: Q2 dependence in individual terms drops out!
    res  = 18*(mH**2 - mu2**2)**2*(mH**2 - mu2**2*cmath.log(mH**2/Q2))/mH**2
    res += 18*(mA**2 - mu2**2)**2*(mA**2 - mu2**2*cmath.log(mA**2/Q2))/mA**2
    res += 48*(mHp**2 - mu2**2)**2*(mHp**2 - mu2**2*cmath.log(mHp**2/Q2))/mHp**2
    res += 6*(2*mA**4*mH**4 - 2*mA**4*mH**2*mu2**2 - 2*mA**2*mH**4*mu2**2 + mA**4*mu2**4 + mH**4*mu2**4 - mA**2*(mH**2 - mu2**2)**2*mu2**2*cmath.log(mA**2/Q2) - mH**2*(mA**2 - mu2**2)**2*mu2**2*cmath.log(mH**2/Q2))/(mH**2*mA**2)
    res += 12*(2*mH**4*mHp**4 - 2*mH**2*mHp**2*(mH**2 + mHp**2)*mu2**2 +(mH**4 + mHp**4)*mu2**4 -mH**2*(mHp**2 - mu2**2)**2*mu2**2*cmath.log(mH**2/Q2) -mHp**2*(mH**2 - mu2**2)**2*mu2**2*cmath.log(mHp**2/Q2))/(mH**2*mHp**2)
    res += 12*(2*mA**4*mHp**4 - 2*mA**2*mHp**2*(mA**2 + mHp**2)*mu2**2 +(mA**4 + mHp**4)*mu2**4 -mA**2*(mHp**2 - mu2**2)**2*mu2**2*cmath.log(mA**2/Q2) -mHp**2*(mA**2 - mu2**2)**2*mu2**2*cmath.log(mHp**2/Q2))/(mA**2*mHp**2)
    res += 6*(mH**2*(1-mu2**2/mH**2)**2+mA**2*(1-mu2**2/mA**2)**2+2*mHp**2*(1-mu2**2/mHp**2)**2)*mu2**2*(math.log(mH**2/Q2)+cmath.log(mA**2/Q2)+4*cmath.log(mHp**2/Q2)-6)
    return l2*res/(256*np.pi**4*vev**3)
def d2lamhhh_IDM_HH_SS_PDOS(mH,mA,mHp,mu2,l2,Q2): # 26.08.2024, new results with mu2 in PDOS scheme for \mu_2
    prod = mH**2*(1 - mu2**2/mH**2)**2 + mA**2*(1 - mu2**2/mA**2)**2 + 2*mHp**2*(1 - mu2**2/mHp**2)**2
    res  = -42*Mt**2*(mH**2-mu2**2)*prod
    res += -12*mH**2*mu2**2*prod*((1-mA**2/mH**2)**3*cmath.log(1-mH**2/mA**2)+2*(1-mHp**2/mH**2)**3*cmath.log(1-mH**2/mHp**2))
    res += 6*mHp**2*(mH**2-mu2**2)*prod*(mH**2/(mH**2-mHp**2)*math.log(mH**2/mHp**2)+mA**2/(mA**2-mHp**2)*math.log(mA**2/mHp**2))
    res += -(mH**2-mu2**2)/(mH**2*mA**2*mHp**2)*prod*(12*mA**6*mHp**2 - 23*mA**4*mH**2*mHp**2 - 2*mH**2*mHp**2*mu2**4 + mA**2*(85*mH**4*mHp**2 + 24*mHp**6 + 46*mHp**2*mu2**4 - 2*mH**2*(23*mHp**4 + 40*mHp**2*mu2**2 + 2*mu2**4)))
    res += 12*l2*vev**2*(mA**4 + 2*mHp**4 - 4*mHp**2*mu2**2 + 3*mu2**4 - (mH**2*mu2**4)/mA**2 - mA**2*(mH**2 + 2*mu2**2) - (2*mH**2*(mHp**4 - 3*mHp**2*mu2**2 + mu2**4))/mHp**2+mu2**2*mA**2*(1-mu2**2/mA**2)**2*math.log(mH**2/mA**2)+2*mu2**2*mHp**2*(1-mu2**2/mHp**2)**2*math.log(mH**2/mHp**2))
    return res.real/(256*np.pi**4*vev**5)
def d2lamhhh_IDM_HH_SSS(mH,mA,mHp,mu2,Q2): # NB: Q2 dependence in individual terms drops out!
    res  = 48*(mH**2 - mu2**2)**4/mH**2
    res += 48*(mA**2 - mu2**2)**4/mA**2
    res += 96*(mHp**2 - mu2**2)**4/mHp**2
    res += 4*(mA**2-mH**2)**2/(mH**6*mA**6)*(3*mA**6*mH**6*(mA**2 + mH**2) - 12*mA**6*mH**6*mu2**2 + 3*mA**4*mH**4*(mA**2 + mH**2)*mu2**4 + mA**2*mH**2*(mA**2 - mH**2)**2*mu2**6 + (mA - mH)*(mA + mH)*(-(mA**6*mu2**2*(-3*mH**4 + mu2**4)*cmath.log(mA**2/Q2)) + mH**6*mu2**2*(-3*mA**4 + mu2**4)*cmath.log(mH**2/Q2) + mA**6*(2*mH**6 - 3*mH**4*mu2**2 + mu2**6)*cmath.log((mA**2 - mH**2)/Q2) - mH**6*(2*mA**6 - 3*mA**4*mu2**2 + mu2**6)*cmath.log((-mA**2 + mH**2)/Q2)))
    res += 8*(mH**2-mHp**2)**2/(mH**6*mHp**6)*(3*mH**6*mHp**6*(mH**2 + mHp**2) - 12*mH**6*mHp**6*mu2**2 + 3*mH**4*mHp**4*(mH**2 + mHp**2)*mu2**4 + mH**2*mHp**2*(mH**2 - mHp**2)**2*mu2**6 + (mH - mHp)*(mH + mHp)*(-(mH**6*mu2**2*(-3*mHp**4 + mu2**4)*cmath.log(mH**2/Q2)) + mHp**6*mu2**2*(-3*mH**4 + mu2**4)*cmath.log(mHp**2/Q2) + mH**6*(2*mHp**6 - 3*mHp**4*mu2**2 + mu2**6)*cmath.log((mH**2 - mHp**2)/Q2) - mHp**6*(2*mH**6 - 3*mH**4*mu2**2 + mu2**6)*cmath.log((-mH**2 + mHp**2)/Q2)))
    res += 8*(mA**2-mHp**2)**2/(mA**6*mHp**6)*(3*mA**6*mHp**6*(mA**2 + mHp**2) - 12*mA**6*mHp**6*mu2**2 + 3*mA**4*mHp**4*(mA**2 + mHp**2)*mu2**4 + mA**2*mHp**2*(mA**2 - mHp**2)**2*mu2**6 + (mA - mHp)*(mA + mHp)*(-(mA**6*mu2**2*(-3*mHp**4 + mu2**4)*cmath.log(mA**2/Q2)) + mHp**6*mu2**2*(-3*mA**4 + mu2**4)*cmath.log(mHp**2/Q2) +  mA**6*(2*mHp**6 - 3*mHp**4*mu2**2 + mu2**6)*cmath.log((mA**2 - mHp**2)/Q2) - mHp**6*(2*mA**6 - 3*mA**4*mu2**2 + mu2**6)*cmath.log((-mA**2 + mHp**2)/Q2)))
    return res.real/(256*np.pi**4*vev**5)
def d2lamhhh_IDM_HH_extlegVEV(mH,mA,mHp,mu2):
    res  = 6*Mt**2*(mH**2-mu2**2)**3/mH**2*(7+3*cw**2/sw**2*_MWMZaEMscheme)
    res += 6*Mt**2*(mA**2-mu2**2)**3/mA**2*(7+3*cw**2/sw**2*_MWMZaEMscheme)
    res += 12*Mt**2*(mHp**2-mu2**2)**3/mHp**2*(7+3*cw**2/sw**2*_MWMZaEMscheme)
    res += 24*Mt**4*(mH**2-mu2**2)**2/mH**2
    res += 24*Mt**4*(mA**2-mu2**2)**2/mA**2
    res += 48*Mt**4*(mHp**2-mu2**2)**2/mHp**2
    res -= 2*((mH**2-mu2*2)**3/mH**2+(mA**2-mu2*2)**3/mA**2+2*(mHp**2-mu2*2)**3/mHp**2)*(mH**2*(1-mu2**2/mH**2)**2+mA**2*(1-mu2**2/mA**2)**2+2*mHp**2*(1-mu2**2/mHp**2)**2)
    return res/(256*np.pi**4*vev**5)
def d2lamhhh_IDM_HH_decoup(mH,mA,mHp,mu2,l2,Q2): # NB: Q2 dependence in individual terms drops out!
    return d2lamhhh_IDM_HH_SS_decoup(mH,mA,mHp,mu2,l2,Q2)+d2lamhhh_IDM_HH_SSS(mH,mA,mHp,mu2,Q2)+d2lamhhh_IDM_HH_extlegVEV(mH,mA,mHp,mu2)
def d2lamhhh_IDM_HH_PDOS(mH,mA,mHp,mu2,l2,Q2): # NB: Q2 dependence in individual terms drops out!
    return d2lamhhh_IDM_HH_SS_PDOS(mH,mA,mHp,mu2,l2,Q2)+d2lamhhh_IDM_HH_SSS(mH,mA,mHp,mu2,Q2)+d2lamhhh_IDM_HH_extlegVEV(mH,mA,mHp,mu2)

@np.vectorize
def kappalam1L(mH,mA,mHp,mu2):
    return 1+(d1lamhhh_SM+d1lamhhh_IDM_HH(mH,mA,mHp,mu2))/lamhhh0L
@np.vectorize
def dkappalam1L_Honly(mH,mA,mHp,mu2):
    return d1lamhhh_IDM_Honly(mH,mA,mHp,mu2)/lamhhh0L
@np.vectorize
def dkappalam1L_Aonly(mH,mA,mHp,mu2):
    return d1lamhhh_IDM_Aonly(mH,mA,mHp,mu2)/lamhhh0L
@np.vectorize
def dkappalam1L_Hpmonly(mH,mA,mHp,mu2):
    return d1lamhhh_IDM_Hpmonly(mH,mA,mHp,mu2)/lamhhh0L
@np.vectorize
def kappalam1L_w_anyH3(mH,mA,mHp,mu2,l2,Q2):
    return kappalam1L_fullIDM(mH,mA,mHp,mu2)
@np.vectorize
def kappalam2L(mH,mA,mHp,mu2,l2,Q2):
    return 1+(d1lamhhh_SM+d1lamhhh_IDM_HH(mH,mA,mHp,mu2)+d2lamhhh_SM+d2lamhhh_IDM_HH_decoup(mH,mA,mHp,mu2,l2,Q2))/lamhhh0L
@np.vectorize
def kappalam2L_w_anyH3(mH,mA,mHp,mu2,l2,Q2):
    return kappalam1L_fullIDM(mH,mA,mHp,mu2)+(d2lamhhh_SM+d2lamhhh_IDM_HH_decoup(mH,mA,mHp,mu2,l2,Q2))/lamhhh0L
@np.vectorize
def kappalam2L_PDOS(mH,mA,mHp,mu2,l2,Q2):
    return 1+(d1lamhhh_SM+d1lamhhh_IDM_HH(mH,mA,mHp,mu2)+d2lamhhh_SM+d2lamhhh_IDM_HH_PDOS(mH,mA,mHp,mu2,l2,Q2))/lamhhh0L
@np.vectorize
def kappalam2L_w_anyH3_PDOS(mH,mA,mHp,mu2,l2,Q2):
    return kappalam1L_fullIDM(mH,mA,mHp,mu2)+(d2lamhhh_SM+d2lamhhh_IDM_HH_PDOS(mH,mA,mHp,mu2,l2,Q2))/lamhhh0L

@np.vectorize
def dkappaf(mH,mA,mHp,mu2):
    lam3 = 2*(mHp**2-mu2**2)/vev**2
    lamH = 2*(mH**2-mu2**2)/vev**2
    lamA = 2*(mA**2-mu2**2)/vev**2
    lam4 = 1/2*(lamH+lamA)-lam3
    lam5 = 1/2*(lamH-lamA)
    res = -(1/4*(lam3 + lam4 + lam5)**2*vev**2*dB0(Mh**2, mH**2, mH**2) + 1/4*(lam3 + lam4 - lam5)**2*vev**2*dB0(Mh**2, mA**2, mA**2) + 1/2*lam3**2*vev**2*dB0(Mh**2, mHp**2, mHp**2))
    return np.real(res)/(16*np.pi**2)

@np.vectorize
def dkappaZ(mH,mA,mHp,mu2):
    lam3 = 2*(mHp**2-mu2**2)/vev**2
    lamH = 2*(mH**2-mu2**2)/vev**2
    lamA = 2*(mA**2-mu2**2)/vev**2
    lam4 = 1/2*(lamH+lamA)-lam3
    lam5 = 1/2*(lamH-lamA)
    w = np.arccos(cw)
    res = 3/2*lam3 + lam4 + 1/2*lam3*np.cos(4*w)
    res += 1/2*(-lam3 - lam4 + lam5)*B0(0, mA**2, mA**2) + 1/2*(lam3 + lam4 - lam5)*B0(0, mA**2, mH**2)
    res -= 1/4*(lam3 + lam4 + lam5)**2*vev**2*dB0(Mh**2, mH**2, mH**2) + 1/4*(lam3 + lam4 - lam5)**2*vev**2*dB0(Mh**2, mA**2, mA**2) + 1/2*lam3**2*vev**2*dB0(Mh**2, mHp**2, mHp**2)
    res += (lam3 + lam4 - lam5)*mA**2*C0(0,0,0,mA**2,mA**2,mH**2) + (lam3 + lam4 + lam5)*mA**2*C0(0,0,0,mA**2,mH**2,mH**2) + 2*lam3*mHp**2*C0(0,0,0,mHp**2,mHp**2,mHp**2)*np.cos(2*w)**2
    return np.real(res)/(16*np.pi**2)

@np.vectorize
def dkappaZsub(mH,mA,mHp,mu2):
    if (np.abs(mH-mHp)< eps) or (np.abs(mA-mHp)< eps):
        res = 0
    elif np.abs(mH-mA) < eps:
        res = (-mH**4 + mHp**4 + 2*mH**2*mHp**2*np.log(mH**2/mHp**2))/(-mH**2 + mHp**2)
    else:    
        res = mHp**2 + (np.log(mH**2/mHp**2)*mH**4*(mA**2 - mHp**2))/((-mA**2 + mH**2)*(mH**2 - mHp**2)) + (np.log(mA**2/mHp**2)*mA**4*(mH**2 - mHp**2))/((mA**2 - mH**2)*(mA**2 - mHp**2))
    return np.real(res*(-1+2*sw**2)/(16*np.pi**2*sw**2*vev**2))

@np.vectorize
def dkappaZkala(S,kappalam):
    prediags = 3/2 - (3*(2*Mh**4 + (MZ**2 - S)**2 - Mh**2*(3*MZ**2 + S))*B0(Mh**2,Mh**2,Mh**2))/(2.*(Mh**4 + (MZ**2 - S)**2 - 2*Mh**2*(MZ**2 + S))) + (3*Mh**2*(Mh**2 - MZ**2 - S)*B0(MZ**2,Mh**2,MZ**2))/(2.*(Mh**4 + (MZ**2 - S)**2 - 2*Mh**2*(MZ**2 + S))) + (3*(Mh**4 - 2*Mh**2*MZ**2 + (MZ**2 - S)**2)*B0(S,Mh**2,MZ**2))/(2.*(Mh**4 + (MZ**2 - S)**2 - 2*Mh**2*(MZ**2 + S))) - 6*MZ**2*C0(Mh**2,MZ**2,S,Mh**2,Mh**2,MZ**2) + (3*Mh**2*(Mh**4 + (MZ**2 - S)**2 - Mh**2*(2*MZ**2 + S))*C0(S,Mh**2,MZ**2,MZ**2,Mh**2,Mh**2))/(Mh**4 + (MZ**2 - S)**2 - 2*Mh**2*(MZ**2 + S))
    return np.real((kappalam-1)*Mh**2/(16*np.pi**2*vev**2)*prediags)

@np.vectorize
def ZZh1L(mH,mA,mHp,mu2,S):
    lam3 = 2*(mHp**2-mu2**2)/vev**2
    lamH = 2*(mH**2-mu2**2)/vev**2
    lamA = 2*(mA**2-mu2**2)/vev**2
    lam4 = 1/2*(lamH+lamA)-lam3
    lam5 = 1/2*(lamH-lamA)
    w = np.arccos(cw)
    res = 1/2*lam3*(3+np.cos(4*w))+lam4
    res -= 1/4*(lam3 + lam4 + lam5)**2*vev**2*dB0(Mh**2, mH**2, mH**2) + 1/4*(lam3 + lam4 - lam5)**2*vev**2*dB0(Mh**2, mA**2, mA**2) + 1/2*lam3**2*vev**2*dB0(Mh**2, mHp**2, mHp**2)
    #
    denom = Mh**4 + (MZ**2 - S)**2 - 2*Mh**2*(MZ**2 + S)
    resb  = -((lam3 + lam4 - lam5)*(2*mA**2*Mh**2 + (MZ**2 - S)**2 - Mh**2*(2*mH**2 + MZ**2 + S))*B0(Mh**2,mA**2,mA**2))/2. 
    resb += ((lam3 + lam4 + lam5)*(2*mA**2*Mh**2 - (MZ**2 - S)**2 + Mh**2*(-2*mH**2 + MZ**2 + S))*B0(Mh**2,mH**2,mH**2))/2.
    resb += + (-(lam5*(mA**2 - mH**2)*(Mh**2 + MZ**2 - S)) - (lam3 + lam4)*MZ**2*(Mh**2 - MZ**2 + S))*B0(MZ**2,mA**2,mH**2) 
    resb += ((-lam3 - lam4)*(Mh**2 + MZ**2 - S)*S - lam5*(mA**2 - mH**2)*(Mh**2 - MZ**2 + S))*B0(S,mA**2,mH**2) 
    resb += (lam3 + lam4 - lam5)*(mA**4*Mh**2 + mH**2*Mh**4 + Mh**2*(mH**2 - MZ**2)*(mH**2 - S) + mA**2*((MZ**2 - S)**2 - Mh**2*(2*mH**2 + MZ**2 + S)))*C0(Mh**2,MZ**2,S,mA**2,mA**2,mH**2) 
    resb += (lam3 + lam4 + lam5)*(mA**4*Mh**2 + Mh**2*(mH**2 - MZ**2)*(mH**2 - S) + mA**2*Mh**2*(-2*mH**2 + Mh**2 - MZ**2 - S) + mH**2*(MZ**2 - S)**2)*C0(S,Mh**2,MZ**2,mA**2,mH**2,mH**2) 
    resb += lam3*(-(MZ**2 - S)**2 + Mh**2*(MZ**2 + S))*B0(Mh**2,mHp**2,mHp**2)*np.cos(2*w)**2 + lam3*MZ**2*(-Mh**2 + MZ**2 - S)*B0(MZ**2,mHp**2,mHp**2)*np.cos(2*w)**2 
    resb -= lam3*(Mh**2 + MZ**2 - S)*S*B0(S,mHp**2,mHp**2)*np.cos(2*w)**2 
    resb += 2*lam3*(Mh**4*mHp**2 + mHp**2*(MZ**2 - S)**2 + Mh**2*(MZ**2*S - 2*mHp**2*(MZ**2 + S)))*C0(Mh**2,MZ**2,S,mHp**2,mHp**2,mHp**2)*np.cos(2*w)**2
    #
    resc = ((1/np.sin(w))**2*(3*mHp**2*A0(mH**2)*(mA**2 - mH**2 + 8*MW**2 - 4*(mA**2 + mH**2 - 2*mHp**2 + 2*MW**2)*np.cos(2*w) - 5*(mA**2 - mH**2)*np.cos(4*w)) + 3*mHp**2*A0(mA**2)*(-mA**2 + mH**2 + 8*MW**2 - 4*(mA**2 + mH**2 - 2*mHp**2 + 2*MW**2)*np.cos(2*w) + 5*(mA**2 - mH**2)*np.cos(4*w)) + 4*(3*A0(mHp**2)*((2*mA**2*mHp**2 + 2*mH**2*mHp**2 - 4*mHp**4 - mHp**2*MW**2 - 4*MW**4)*np.cos(2*w) + MW**2*(2*mHp**2 + 3*MW**2 + (2*mHp**2 + MW**2)*np.cos(4*w) - 3*mHp**2*np.cos(6*w))) - 4*mHp**2*MW**2*(3*mA**2 + 3*mH**2 + 9*mHp**2 + 5*MW**2 + (12*mHp**2 - 5*MW**2)*np.cos(2*w) + 9*mHp**2*np.cos(4*w))*np.sin(w)**2)))/(144.*mHp**2*MW**2)
    resc += ((mA**4 + (mHp**2 - MW**2)**2 - 2*mA**2*(mHp**2 + MW**2))*B0(MW**2,mA**2,mHp**2)*np.cos(2*w)*(1/np.sin(w))**2)/(6.*MW**2) 
    resc += ((mH**4 + (mHp**2 - MW**2)**2 - 2*mH**2*(mHp**2 + MW**2))*B0(MW**2,mH**2,mHp**2)*np.cos(2*w)*(1/np.sin(w))**2)/(6.*MW**2) 
    resc += (B0(MZ**2,mHp**2,mHp**2)*np.cos(2*w)**2*(-2*mHp**2 - MW**2 + 6*mHp**2*np.cos(2*w))*(1/np.sin(w))**2)/6. 
    resc -= (B0(MZ**2,mA**2,mH**2)*(-mA**4 + 2*mA**2*mH**2 - mH**4 + 8*mA**2*MW**2 + 8*mH**2*MW**2 + 8*MW**4 + 4*(mA**4 + mH**4 - 6*mH**2*MW**2 - 2*mA**2*(mH**2 + 3*MW**2))*np.cos(2*w) + 5*(mA**2 - mH**2)**2*np.cos(4*w))*(1/np.sin(w))**2)/(48.*MW**2) 
    resc -= (MW**2*dB0(MZ**2,mHp**2,mHp**2)*np.cos(2*w)**2*(1/np.cos(w))**2*(-4*mHp**2 + MZ**2))/3. 
    resc += (dB0(MZ**2,mA**2,mH**2)*(-(mA**2 - mH**2)**2 + 2*(mA**2 + mH**2)*MZ**2 - MZ**4))/3.
    return np.real(res+resb/denom+resc/vev**2)/(16*np.pi**2)
    

In [4]:
def quartconv(mH,mA,mHp,lam345,lam2,mu2):
    lam345bar = 2*(mA**2-mu2**2)/vev**2
    lam3 = 2*(mHp**2-mu2**2)/vev**2
    lam4 = 1/2*(lam345+lam345bar)-lam3
    lam5 = 1/2*(lam345-lam345bar)
    return lam1, lam2, np.real(lam3), np.real(lam4), np.real(lam5)

def checkinertvacuum(mu2,lam2):
    if np.real(mu2**2)/np.sqrt(lam2) >= -1/2*Mh*vev:
        return True
    else:
        return False
    
def checkBFB(lam1,lam2,lam3,lam4,lam5):

    if (lam1 >= 0) and (lam2 >= 0) and (np.sqrt(lam1*lam2)+lam3+min(0,lam4+lam5,lam4-lam5)):
        return True
    else: 
        return False
    
def generate_point():
    mHrand = random.uniform(mHmin, mHmax)
    
    # Generate 'b' between 'a' and 10
    mArand = random.uniform(mHrand, mAmax)
    
    # Generate 'c' within 10% of 'b'
    deviation = 0.1  # 10% of b
    mHprand = random.uniform(mArand*(1 - deviation), mArand*(1 + deviation))
    
    lam345rand = random.uniform(-1,1)
    lam2rand = random.uniform(0,4*np.pi)

    mu2 = cmath.sqrt(mHrand**2-1/2*lam345rand*vev**2)

    return mHrand, mArand, mHprand, lam345rand, lam2rand, mu2

In [5]:
allpoints = pd.read_csv('./allpointsdata_scanIDM_2024-12-11_19-39-22.csv',index_col=0)
#allpoints = pd.read_csv('./allpointsdata_scanIDM_2024-12-06_11-58-49.csv',index_col=0)

In [6]:
allpoints['kagam2LDI'] = kappagam2L(allpoints['mHp'],allpoints['mH'],allpoints['mA'],np.sqrt(allpoints['mu2sq']),allpoints['lam2'])
allpoints['kalam2LDI'] = np.real(kappalam2L(allpoints['mH'],allpoints['mA'],allpoints['mHp'],np.sqrt(allpoints['mu2sq']),allpoints['lam2'],300**2))

/home/braathen/.local/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/braathen/.local/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [8]:
allpoints['MW1L'] = ewpo.MW1L_vec(allpoints['mH'],allpoints['mA'],allpoints['mHp'],allpoints['lam345'])
allpoints['sl1L'] = ewpo.sl1L_vec(allpoints['mH'],allpoints['mA'],allpoints['mHp'],allpoints['lam345'])
allpoints['GamZ1L'] = ewpo.GamZ1L_vec(allpoints['mH'],allpoints['mA'],allpoints['mHp'],allpoints['lam345'])
allpoints['MW2L'] = ewpo.MW2L_vec(allpoints['mH'],allpoints['mA'],allpoints['mHp'],allpoints['lam345'])
allpoints['sl2L'] = ewpo.sl2L_vec(allpoints['mH'],allpoints['mA'],allpoints['mHp'],allpoints['lam345'])
allpoints['GamZ2L'] = ewpo.GamZ2L_vec(allpoints['mH'],allpoints['mA'],allpoints['mHp'],allpoints['lam345'])

allpoints['effZZh1L_s0'] = ZZh1L(allpoints['mH'],allpoints['mA'],allpoints['mHp'],np.sqrt(allpoints['mu2sq']),0)
allpoints['effZZh1L_s240'] = ZZh1L(allpoints['mH'],allpoints['mA'],allpoints['mHp'],np.sqrt(allpoints['mu2sq']),240**2)
allpoints['effZZh1L_s365'] = ZZh1L(allpoints['mH'],allpoints['mA'],allpoints['mHp'],np.sqrt(allpoints['mu2sq']),365**2)
allpoints['effZZh1L_s500'] = ZZh1L(allpoints['mH'],allpoints['mA'],allpoints['mHp'],np.sqrt(allpoints['mu2sq']),500**2)
allpoints['effZZh1L_s550'] = ZZh1L(allpoints['mH'],allpoints['mA'],allpoints['mHp'],np.sqrt(allpoints['mu2sq']),550**2)

allpoints['effZZh1Lkala_s0'] = dkappaZkala(0,allpoints['kalam1L'])
allpoints['effZZh1Lkala_s240'] = dkappaZkala(240**2,allpoints['kalam1L'])
allpoints['effZZh1Lkala_s365'] = dkappaZkala(365**2,allpoints['kalam1L'])
allpoints['effZZh1Lkala_s500'] = dkappaZkala(500**2,allpoints['kalam1L'])
allpoints['effZZh1Lkala_s550'] = dkappaZkala(550**2,allpoints['kalam1L'])

allpoints['dkappaf'] = dkappaf(allpoints['mH'],allpoints['mA'],allpoints['mHp'],np.sqrt(allpoints['mu2sq']))

allpoints['DeltaGammagamgam1L'] = DeltaGammagamgam1L(allpoints['mHp'],np.sqrt(allpoints['mu2sq']))
allpoints['DeltaGammagamgam2L'] = DeltaGammagamgam2LPDOS(allpoints['mHp'],allpoints['mH'],allpoints['mA'],np.sqrt(allpoints['mu2sq']),allpoints['lam2'])
allpoints['DeltaGammagamgam2LDI'] = DeltaGammagamgam2L(allpoints['mHp'],allpoints['mH'],allpoints['mA'],np.sqrt(allpoints['mu2sq']),allpoints['lam2'])

allpoints['dkappaZgam1L'] = kappagamZ1L(allpoints['mHp'],np.sqrt(allpoints['mu2sq']))
allpoints['DeltaGammaZgam1L'] = DeltaGammaZgam(allpoints['mHp'],np.sqrt(allpoints['mu2sq']))


/home/braathen/.local/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/braathen/.local/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/braathen/.local/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/braathen/.local/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/braathen/.local/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/braathen/.local/lib/python3.11/site-packages/pandas/core/arraylike.py:399:

TypeError: unhashable type: 'Series'

In [12]:
allpoints['DeltaGammaZgam1L'] = DeltaGammaZgam(allpoints['mHp'],np.sqrt(allpoints['mu2sq']))

In [14]:
allpoints.to_csv('./allIDMpoints_2025-01-04.csv')
#allpoints.to_csv('./allIDMpoints_2024-12-15.csv')

In [13]:
allpoints

,mH,mA,mHp,lam345,lam2,mu2sq,lam1,lam3,lam4,lam5,...,effZZh1Lkala_s240,effZZh1Lkala_s365,effZZh1Lkala_s500,effZZh1Lkala_s550,dkappaf,DeltaGammagamgam1L,DeltaGammagamgam2L,DeltaGammagamgam2LDI,dkappaZgam1L,DeltaGammaZgam1L
0,931.934503,1113.358965,1118.289211,-0.673010,4.317180,8.889022e+05,0.258148,11.931507,-6.483747,-6.120770,...,0.008112,0.002892,0.000722,0.000171,-0.005391,-0.029509,-0.038609,-0.042257,-0.550302,-0.010976
1,1063.833836,1171.316886,1194.458854,0.273193,11.811667,1.123461e+06,0.258148,10.004948,-5.768962,-3.962793,...,0.003389,0.001208,0.000302,0.000071,-0.003036,-0.021727,-0.028100,-0.032509,-0.404365,-0.008071
2,75.826959,442.165167,446.657856,-0.133518,9.134022,9.796947e+03,0.258148,6.258443,-3.261849,-3.130113,...,0.007375,0.002630,0.000657,0.000155,-0.009620,-0.096189,-0.109315,-0.123478,-1.829814,-0.036261
3,681.874809,970.153663,973.494062,0.363153,7.242954,4.539453e+05,0.258148,16.288744,-8.069891,-7.855700,...,0.029032,0.010351,0.002585,0.000612,-0.013397,-0.052868,-0.073421,-0.083873,-0.992040,-0.019742
4,946.942768,1170.073303,1170.727146,0.237104,3.331870,8.895135e+05,0.258148,15.871194,-7.842291,-7.791799,...,0.018524,0.006605,0.001649,0.000390,-0.008825,-0.035752,-0.050945,-0.055541,-0.667778,-0.013311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,746.520239,1069.403338,1074.688230,-0.600766,5.589704,5.755029e+05,0.258148,19.116217,-10.045402,-9.671581,...,0.038491,0.013724,0.003427,0.000811,-0.015087,-0.050917,-0.071589,-0.081949,-0.954833,-0.019005
1996,734.712492,871.135309,877.965018,-0.177533,12.440877,5.451838e+05,0.258148,7.443861,-4.007745,-3.613649,...,0.002791,0.000995,0.000249,0.000059,-0.003362,-0.029896,-0.036252,-0.042648,-0.557739,-0.011124
1997,891.012784,1047.231070,1053.927279,-0.472568,3.782147,8.082283e+05,0.258148,9.980662,-5.458698,-4.994533,...,0.005074,0.001809,0.000452,0.000107,-0.004206,-0.027809,-0.035203,-0.038060,-0.518404,-0.010341
1998,242.791319,770.461437,773.249771,-0.048518,9.698190,6.041830e+04,0.258148,17.732115,-8.961318,-8.819315,...,0.060295,0.021498,0.005369,0.001270,-0.025295,-0.090437,-0.119333,-0.142320,-1.714511,-0.033996
